In [5]:
pip install scapy numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444347 sha256=ddda180bd44f95105d7853243c247c1b2584b5d32f86719245fb90fc9efd5da6
  Stored in directory: /home/seojin929_gmail_com/.cache/pip/wheels/82/b7/03/8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully built scapy
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install numpy networkx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import networkx as nx
import numpy as np
from collections import defaultdict

# --- Parameters Setup ---
CHUNK_LENGTH = 50  # Length of shingles to be used when hashing the graphs.
L = 1000  # Number of hash functions (sketch size).
SEED = 23  # Seed for random number generation to ensure reproducibility.
CLUSTER_UPDATE_INTERVAL = 10000  # Number of edges after which clusters are updated.
B = 100  # Number of hash bands used in Locality-Sensitive Hashing (not used here, but part of LSH).
R = 20  # Number of rows in each band in LSH.
PI = 3.1415926535897  # Approximate value of PI, could be used in similarity computations (but not in this script).

# --- Initialize Random Vectors for StreamHash ---
np.random.seed(SEED)  # Set the random seed to make the process reproducible.
MAX_UINT64 = np.iinfo(np.uint64).max  # Get the maximum value for an unsigned 64-bit integer (used for random hash generation).
# Generate L random vectors, each with CHUNK_LENGTH + 2 elements (64-bit integers). This serves as the universal hash family H.
H = [np.random.randint(0, MAX_UINT64, CHUNK_LENGTH+2, dtype=np.uint64) for _ in range(L)]


# --- Function to Hash Shingles ---
# This function creates a hash for each shingle (a substructure of the graph). Shingles are combinations of node and edge types.
def hash_shingle(shingle, randbits):
    # Initialize sum_hash as the first random bit.
    sum_hash = int(randbits[0])
    # For each character in the shingle, update the sum_hash using a unique random number from randbits.
    for i, char in enumerate(shingle):
        sum_hash += int(randbits[i+1]) * ord(char)  # Multiply character code (ord) with a random bit.
    
    # The result is right-shifted by 63 to get the most significant bit (which we use as a hash).
    # If the MSB is 1, return +1; if 0, return -1 (this gives a binary hash).
    return 2 * ((sum_hash >> 63) & 1) - 1


# --- Function to Create a StreamHash Sketch for a Graph ---
# Constructs a sketch (a compact representation) of a graph using StreamHash. Each graph is represented by its set of shingles.
def construct_streamhash_sketch(shingle_vector):
    projection = np.zeros(L, dtype=int)  # Initialize the projection vector (length L) for the graph's shingle hashes.
    
    # For each shingle in the graph, apply the hash function and accumulate its contribution in the projection vector.
    for shingle, count in shingle_vector.items():
        for i in range(L):
            projection[i] += count * hash_shingle(shingle, H[i])  # Multiply shingle occurrence with its hash value.
    
    # Convert the projection into a binary sketch (0s and 1s). If projection[i] >= 0, set sketch[i] to 1; else, set it to 0.
    sketch = np.where(projection >= 0, 1, 0)
    return sketch, projection  # Return both the sketch and the projection vector for further use.


# --- Function to Compute Similarity Between Two Sketches ---
# Compares two sketches by computing the fraction of matching bits (similarity score between 0 and 1).
def streamhash_similarity(sketch1, sketch2):
    return np.sum(sketch1 == sketch2) / L  # Compute how many bits are the same and divide by the sketch length.


# --- Function to Process Edges Into Graphs ---
# Reads an edge file and constructs graphs from it. Each graph is a collection of nodes connected by edges.
def process_edges(edge_file):
    graphs = defaultdict(nx.DiGraph)  # Initialize a dictionary to hold multiple directed graphs (DiGraph).
    
    # Open the edge file and read it line by line. Each line represents an edge between two nodes in a graph.
    with open(edge_file, 'r') as f:
        for line in f:
            # Extract source node, source type, destination node, destination type, edge type, and graph ID from the line.
            src_id, src_type, dst_id, dst_type, e_type, gid = line.strip().split()
            # Add an edge between src_id and dst_id in the graph gid.
            graphs[int(gid)].add_edge((src_id, src_type), (dst_id, dst_type), e_type=e_type)
    
    return graphs  # Return a dictionary of graphs.


# --- Function to Create Shingle Vectors for Each Graph ---
# Constructs shingle vectors for each graph, where each shingle is a combination of source type, edge type, and destination type.
def construct_shingle_vectors(graphs):
    shingle_vectors = {}  # Initialize a dictionary to store the shingle vector for each graph.
    
    # For each graph, iterate over its edges and construct shingles.
    for gid, graph in graphs.items():
        shingle_vector = defaultdict(int)  # Initialize a dictionary to store shingles and their counts.
        
        # For each edge in the graph, extract the source type, edge type, and destination type to form a shingle.
        for u, v, data in graph.edges(data=True):
            src_type, dst_type = u[1], v[1]  # Get the types of the source and destination nodes.
            shingle = f"{src_type}{data['e_type']}{dst_type}"  # Create the shingle as a string.
            shingle_vector[shingle] += 1  
        # Store the shingle vector for the current graph.
        shingle_vectors[gid] = shingle_vector  
    # Return a dictionary of shingle vectors for all graphs.
    return shingle_vectors  


# --- Function to Bootstrap Clusters ---
# Reads the initial clusters and thresholds from a file to initialize clusters for the anomaly detection.
def read_bootstrap_clusters(bootstrap_file):
    # Initialize a dictionary to hold clusters (lists of graph IDs).
    clusters = defaultdict(list)  
    
    # Open the bootstrap cluster file and read the first line to get the number of clusters and the global threshold.
    with open(bootstrap_file, 'r') as f:
        # Global threshold is used for anomaly detection.
        nclusters, global_threshold = map(float, f.readline().split())  
        
        # Read the subsequent lines to map each graph to its corresponding cluster.
        for line in f:
            # Read the cluster-specific threshold and graph ID.
            threshold, gid = map(float, line.split())  
            # Add the graph ID to the cluster corresponding to this threshold.
            clusters[int(threshold)].append(int(gid))  
    # Return the clusters and the global threshold.
    return clusters, global_threshold  


# --- Anomaly Detection ---
# Computes the distance between each graph's sketch and the cluster centroids, and flags anomalies if the distance is too high.
def update_distances_and_clusters(gid, graph_sketches, centroid_sketches, global_threshold):
     # Retrieve the sketch for the graph.
    sketch = graph_sketches[gid] 
    # Initialize the minimum distance as the maximum possible (1.0).
    min_distance = 1.0  
     # Initialize the nearest cluster as None
    nearest_cluster = None 
    
    # Iterate through each cluster and compute the similarity between the graph's sketch and the cluster's centroid sketch.
    for cluster_id, centroid_sketch in centroid_sketches.items():
        # Compute the similarity (between 0 and 1).
        sim = streamhash_similarity(sketch, centroid_sketch)  
         # Convert similarity to distance (closer to 0 is better).
        distance = 1.0 - sim 
        
        # If this distance is smaller than the current minimum, update the minimum distance and the nearest cluster.
        if distance < min_distance:
            min_distance = distance
            nearest_cluster = cluster_id
    
    # If the distance is greater than the global threshold, the graph is flagged as an anomaly.
    if min_distance > global_threshold:
        print(f"Graph {gid} is an anomaly with score: {min_distance}")
    else:
        # Otherwise, the graph is assigned to the nearest cluster.
        print(f"Graph {gid} assigned to cluster {nearest_cluster} with score: {min_distance}")


# --- Main Function ---
def main():
    # Step 1: Read the edge file and bootstrap clusters.
    # Load graphs from the edge file.
    graphs = process_edges('test_edges.txt')  
    # Load clusters and threshold.
    clusters, global_threshold = read_bootstrap_clusters('test_bootstrap_clusters.txt')  

    # Step 2: Create shingle vectors and StreamHash sketches for all graphs.
    # Construct shingles for each graph.
    shingle_vectors = construct_shingle_vectors(graphs)  
    # Initialize a dictionary to hold the sketches for each graph.
    graph_sketches = {}  
    
    # For each graph, generate its StreamHash sketch from the shingle vector.
    for gid, shingle_vector in shingle_vectors.items():
         # Create the sketch for the graph.
        sketch, projection = construct_streamhash_sketch(shingle_vector) 
        # Store the sketch in the dictionary
        graph_sketches[gid] = sketch  .
    
    # Step 3: Initialize cluster centroids (average sketches for each cluster).
    # Dictionary to store centroid sketches for each cluster.
    centroid_sketches = {}  
    for cluster_id, gids in clusters.items():
        # Compute average projection.
        centroid_projection = np.mean([graph_sketches[gid] for gid in gids], axis=0)  
        # Convert average projection to a binary sketch.
        centroid_sketch = np.where(centroid_projection >= 0, 1, 0)  
         # Store the centroid sketch.
        centroid_sketches[cluster_id] = centroid_sketch 
    
    # Step 4: Perform anomaly detection for all graphs.
    for gid in graph_sketches.keys():
        update_distances_and_clusters(gid, graph_sketches, centroid_sketches, global_threshold)  # Detect anomalies.

# --- Entry Point ---
if __name__ == "__main__":
    # Execute the main function when the script is run.
    main()  


Graph 0 assigned to cluster 0 with score: 0.504
Graph 1 assigned to cluster 0 with score: 0.33599999999999997
Graph 2 assigned to cluster 0 with score: 0.33599999999999997
Graph 3 assigned to cluster 0 with score: 0.505
